In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128            # 생성자의 입력값으로 사용할 노이즈 크기입니다.


X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 구분자는 진짜와 얼마나 가까운가를 판단하는 값으로, 0~1사이의 값을 출력할 것입니다.
# 따라서 하나의 스칼라값을 출력하도록 구성했습니다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

# GAN의 학습은 loss_D와 loss_G 모두를 최대화하는 것입니다.
# 다만 loss_D와 loss_G는 서로 연관되어 있어서 두 손실값이 항상 같이 증가하는 경향을 보이지는 않을 것입니다.

# 진짜 이미지 판별값 D_real은 1에 가까워야 하고(진짜라고 판별),
# 가짜 이미지 판별값 D_gene는 0에 가까워야 합니다(가짜라고 판별)
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

# 가짜 이미지 판별값 D_gene를 1에 가깝게 만들기만 하면 됩니다.
loss_G = tf.reduce_mean(tf.log(D_gene))

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# GAN논문에 따르면 loss를 최대화해야 합니다만,
# 최적화에 쓸 수 있는 함수는 minimize뿐이므로 최적화하려는 loss_D와 loss_G에 음수 부호를 붙여줬습니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

/usr/local/Cellar/python3/3.6.4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
    
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
    
print('최적화 완료!')

Epoch: 0000 D loss: -0.4999 G loss: -2.161
Epoch: 0001 D loss: -0.4316 G loss: -2.197
Epoch: 0002 D loss: -0.2822 G loss: -2.3
Epoch: 0003 D loss: -0.5544 G loss: -1.37
Epoch: 0004 D loss: -0.4493 G loss: -1.845
Epoch: 0005 D loss: -0.3712 G loss: -2.286
Epoch: 0006 D loss: -0.188 G loss: -2.9
Epoch: 0007 D loss: -0.1991 G loss: -2.994
Epoch: 0008 D loss: -0.3032 G loss: -2.541
Epoch: 0009 D loss: -0.3599 G loss: -2.322
Epoch: 0010 D loss: -0.3444 G loss: -2.47
Epoch: 0011 D loss: -0.5922 G loss: -1.996
Epoch: 0012 D loss: -0.3625 G loss: -2.438
Epoch: 0013 D loss: -0.5096 G loss: -2.078
Epoch: 0014 D loss: -0.4609 G loss: -2.357
Epoch: 0015 D loss: -0.4457 G loss: -2.411
Epoch: 0016 D loss: -0.3604 G loss: -2.294
Epoch: 0017 D loss: -0.5404 G loss: -2.326
Epoch: 0018 D loss: -0.5456 G loss: -2.215
Epoch: 0019 D loss: -0.4516 G loss: -2.261
Epoch: 0020 D loss: -0.6573 G loss: -1.833
Epoch: 0021 D loss: -0.644 G loss: -2.144
Epoch: 0022 D loss: -0.4205 G loss: -2.155
Epoch: 0023 D loss: